# FrozenLake-v0

In [1]:
import numpy as np
import gym
from api_key import api_key

Walking through and understanding code of : https://medium.com/emergent-future/simple-reinforcement-learning-with-tensorflow-part-0-q-learning-with-tables-and-neural-networks-d195264329d0#.7cwz7def1

In [51]:
env = gym.make('FrozenLake-v0')
obs = env.reset()
obs

env.render()

[2016-12-19 23:12:53,908] Making new env: FrozenLake-v0


SFFF
FHFH
FFFH
HFFG



## Completely random - how many 

In [3]:
reward_list = []

for a in range(10000):
    curr_state = env.reset()
    for t in range(100):

        ## Take a random action
        action = env.action_space.sample()

        ## take steps
        new_state, reward, done, info = env.step(action)
        
        if done:
            reward_list.append(reward)
            break
        
        # current step for next iteration
        curr_state = new_state
    
print 'Final reward: ', sum(reward_list)/len(reward_list)

Final reward:  0.013


## Using Q-table learning

$$ {\displaystyle Q(s_{t},a_{t})\leftarrow \underbrace {Q(s_{t},a_{t})} _{\rm {old~value}}+\underbrace {\alpha } _{\rm {learning~rate}}\cdot \left(\overbrace {\underbrace {r_{t+1}} _{\rm {reward}}+\underbrace {\gamma } _{\rm {discount~factor}}\cdot \underbrace {\max _{a}Q(s_{t+1},a)} _{\rm {estimate~of~optimal~future~value}}} ^{\rm {learned~value}}-\underbrace {Q(s_{t},a_{t})} _{\rm {old~value}}\right)} $$

In [111]:
##Formalise the functions for each update and next action
def update_q(q,curr_state,new_state,action,reward,learning_rate,discount_factor):
    
    learned_value = reward + discount_factor*np.max(q[new_state,:])
    #q[curr_state,action] = q[curr_state,action] + learning_rate*(learned_value-q[curr_state,action])
    
    q[curr_state,action] = q[curr_state,action] + learning_rate*(learned_value - q[curr_state,action])

    return q

def select_action(q,curr_state,rand_scale):
    return np.argmax(q[curr_state,:] + np.random.randn(1,env.action_space.n)*rand_scale)

In [112]:
#env.monitor.start('/tmp/FrozenLake-v0',force=True)

Q = np.zeros([env.observation_space.n,env.action_space.n])

discount_factor = 0.99
learning_rate = 0.8

reward_list = []

episodes = 10000

for a in range(episodes):
    curr_state = env.reset()
    full_reward = 0
    for t in range(100):

        ## Take a random action
        rand_scale = (1./(a+1))
        action = select_action(Q,curr_state,rand_scale)
        
        ## take steps
        new_state, reward, done, info = env.step(action)
        
        ##Update Q based on steps
        full_reward += reward
        #learning_rate = 1.-(1./(a+1))
        Q = update_q(Q,curr_state,new_state,action,reward,learning_rate,discount_factor)
        
        if done:
            reward_list.append(full_reward)
            break
        
        # current step for next iteration
        curr_state = new_state
    
print 'Final reward: ', sum(reward_list)/len(reward_list)

#env.monitor.close()
#gym.upload('/tmp/FrozenLake-v0',api_key=api_key)

Final reward:  0.759324569841
